[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/andresfcm97/ia4eng_afcm_eac_proyecto/blob/main/03_Modelos_e_Iteraciones.ipynb)

In [1]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import google.colab as gc
gc.files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"andresfelipeceron","key":"e992d419fad80c7317b2e47100fead4e"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c house-prices-advanced-regression-techniques
from zipfile import ZipFile
file_name = 'house-prices-advanced-regression-techniques.zip'
with ZipFile(file_name, 'r') as zip:
  zip.extractall()

  0% 0.00/199k [00:00<?, ?B/s]
100% 199k/199k [00:00<00:00, 60.1MB/s]


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm
from sklearn.model_selection import train_test_split

entrenamiento = pd.read_csv("train.csv")
prueba = pd.read_csv("test.csv")
ID_prueba = prueba['Id']

correlacion = entrenamiento.corr()
coefi_correla = correlacion['SalePrice'].sort_values(ascending=False)
correla_aceptable = 0.3
columnas_nuevo = correlacion[correlacion['SalePrice'] > correla_aceptable].index
columnas_entrenamiento = [i for i in columnas_nuevo]
columnas_prueba = [columnas_nuevo[i] for i in range(len(columnas_nuevo) - 1)]

df_entrenamiento = entrenamiento[columnas_entrenamiento]
df_prueba = prueba[columnas_prueba]

total = df_entrenamiento.isnull().sum().sort_values(ascending=False)
porcentaje = (df_entrenamiento.isnull().sum()/df_entrenamiento.isnull().count()).sort_values(ascending=False)
datos_faltantes = pd.concat([total, porcentaje], axis=1, keys=['Cant. Nulos', 'Porcentaje'])
df_entrenamiento = df_entrenamiento.drop((datos_faltantes[datos_faltantes['Cant. Nulos'] > 81]).index,1)
total_prueba = df_prueba.isnull().sum().sort_values(ascending=False)
porcentaje_prueba = (df_prueba.isnull().sum()/df_prueba.isnull().count()).sort_values(ascending=False)
datos_faltantes = pd.concat([total_prueba, porcentaje_prueba], axis=1, keys=['Cant. Nulos', 'Porcentaje'])
df_prueba = df_prueba.drop((datos_faltantes[datos_faltantes['Cant. Nulos'] > 78]).index,1)

df_entrenamiento['GarageYrBlt'] = df_entrenamiento['GarageYrBlt'].fillna(df_entrenamiento['GarageYrBlt'].mean())
df_entrenamiento['MasVnrArea'] = df_entrenamiento['MasVnrArea'].fillna(df_entrenamiento['MasVnrArea'].mean())

df_prueba['GarageYrBlt'] = df_prueba['GarageYrBlt'].fillna(df_prueba['GarageYrBlt'].mean())
df_prueba['MasVnrArea'] = df_prueba['MasVnrArea'].fillna(df_prueba['MasVnrArea'].mean())

df_entrenamiento['SalePrice'] = np.log(df_entrenamiento['SalePrice'])

df_entrenamiento = pd.get_dummies(df_entrenamiento)
df_prueba = pd.get_dummies(df_prueba)

for col in ( 'GarageArea', 'GarageCars'):
    df_prueba[col] = df_prueba[col].fillna(0)
for col in ('BsmtFinSF1','TotalBsmtSF'):
    df_prueba[col] = df_prueba[col].fillna(0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


Se procede a sacar las variables de entrenamiento y prueba

In [5]:
y_train = df_entrenamiento.SalePrice.values
df_entrenamiento.drop("SalePrice", axis = 1, inplace = True)
test_pct = 0.2
# Con la siguiente línea, de los datos df_entrenamiento y de y_train, serán divididos aleatoriamente de
# tal forma que el 20% correspondrá a test y el 80% a train
X_train, X_test, y_train, y_test = train_test_split(df_entrenamiento, y_train, test_size=test_pct)

In [6]:
y_train

array([12.67607627, 11.6127708 , 11.65268741, ..., 12.03171926,
       12.14950229, 11.82041016])

In [7]:
X_train

,OverallQual,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,FullBath,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF
159,7,2005,2006,456.0,57,1392,1392,1070,2462,2,9,1,2006.000000,2,576,239,132
1063,6,1925,1980,0.0,397,1103,1103,0,1103,1,5,1,1976.000000,2,440,166,120
140,4,1971,1971,0.0,432,864,864,0,864,1,5,1,1978.506164,0,0,0,0
104,7,1931,1950,600.0,224,1040,1226,592,1818,1,7,2,1951.000000,1,240,0,0
1357,5,1971,2008,0.0,734,1078,1078,0,1078,1,6,1,1971.000000,2,500,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978,4,1954,1954,0.0,552,894,894,0,894,1,5,0,1999.000000,2,400,0,0
1071,6,1968,1968,0.0,298,727,829,727,1556,1,8,0,1968.000000,2,441,0,0
404,6,1995,1996,0.0,0,806,806,766,1572,2,7,1,1995.000000,2,373,0,40
827,7,2001,2001,0.0,20,1454,1434,0,1434,2,6,1,2001.000000,2,527,290,39


# Modelos

In [8]:
from sklearn.linear_model import LogisticRegression
#from sklearn.ensemble import RandomForestClassifier
model = LogisticRegression()
model.fit(X_train, y_train)
y_predic = model.predict(X_test)
y_predic

ValueError: ignored

Tal como se puede apreciar se genera un error ya que, el estimador que se intentó utilizar se usa para predicciones lineales y no continuas, por tanto, a continuación se investigaron algunos estimadores que se utilizan para realizar predicciones tipo continuas, como lo es para nuestro ejercicio.

En la siguiente celda, se invocan las librerías que se van a utilizar

In [9]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, BaggingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn.svm import SVR
from mlxtend.regressor import StackingCVRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

En la siguiente celda estaremos llamando los estimadores que se utilizarán, junto con los parámetros necesarios para utilizarlos

In [10]:
# Se instancia DecisionTreeRegressor
from sklearn.tree import DecisionTreeRegressor
DTR = DecisionTreeRegressor(random_state = 100)

# Se instancia Gradient Boosting Regression
from sklearn.ensemble import GradientBoostingRegressor
params = {'n_estimators': 150, 'max_depth': 5, 'min_samples_split': 2,
          'learning_rate': 0.05, 'loss': 'ls'}

GBR = GradientBoostingRegressor(**params)

# Se instancia svr
from sklearn.svm import SVR
svr = SVR(kernel = 'rbf')

# Se instancia random forest regressor
from sklearn.ensemble import RandomForestRegressor
RFR = RandomForestRegressor(n_estimators = 100, random_state = 0)


# Se instancia XGBRegressor
import xgboost as xgb
XGBR = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)

# Se instancia LightGBM
import lightgbm as lgb
LGBMR = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.1, n_estimators=500,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)



# Se instancia Lasso
from sklearn.linear_model import Lasso
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))

Creamos una lista que contiene a los estimadores recién creados

In [11]:
classifiers = [
    ('DecisionTreeRegressor', DTR),
    ('Gradient Boosting Regression', GBR),
    ('XGBRegressor', XGBR),
    ('Lasso',lasso),
    ('RandomForestRegressor', RFR),
    ('Support Vector Machine Regression', svr),
    ('LightGBM',LGBMR)
]

Para calcular el error de cada estimador, importamos las siguientes librerías, además, se itera con los elementos de la lista para calcular los datos predichos usando cada estimador y calculando su respectivo error, en este caso, el error cuadrático medio

In [12]:
from sklearn.metrics import accuracy_score
from sklearn import metrics
# Se itera sobre la lista de clasificadores creada
for nombre_modelo, modelo in classifiers:
    # Se fitea el modelo al conjunto de entrenamiento
    modelo.fit(X_train, y_train)
    
    # Se predice y
    y_predic = modelo.predict(X_test)
    y_predic = y_predic.reshape(-1,1)
    # Se calcula la exactitud con el RMSE
    error = np.sqrt(metrics.mean_squared_error(y_test, y_predic))
    # Se evalúa la exactitud del modelo en conjunto con los datos de prueba
    print('{:s} RMSE : {:.3f}'.format(nombre_modelo, error))

DecisionTreeRegressor RMSE : 0.213
Gradient Boosting Regression RMSE : 0.144
XGBRegressor RMSE : 0.138
Lasso RMSE : 0.146
RandomForestRegressor RMSE : 0.144
Support Vector Machine Regression RMSE : 0.174
LightGBM RMSE : 0.137


Tal como se puede observar, tanto el Gradient Boosting Regression y el LightGBM, presentan el mínimo error cuadrático medio (MSE), por tanto, trabajaremos con ellos. Después de correr varias veces el programa, se verificó que estos dos siempre presentaron el menor error con respecto a los otros, sin embargo, el que presentó un menor error entre estos dos fue principalmente el Gradient Boosting Regression, por tanto, se usará un 80% de los resultados del Gradient Boosting Regression y un 20% de los resultados del LigthGBM.

In [13]:
GBR.fit(X_train, y_train)

GradientBoostingRegressor(learning_rate=0.05, loss='ls', max_depth=5,
                          n_estimators=150)

In [15]:
LGBMR_predic = np.expm1(LGBMR.predict(df_prueba.values))
GBR_predic = np.expm1(GBR.predict(df_prueba.values))
Estimador_final_predic = LGBMR_predic*0.2 + GBR_predic*0.8

In [16]:
Estimador_final_predic

array([126125.42904309, 153052.89098995, 183451.71961204, ...,
       166293.98574224, 110803.53374314, 229844.5075752 ])

Convertimos este resultado a un DataFrame para mejorar su visualización

In [17]:
df_Estimador_final = pd.DataFrame(Estimador_final_predic, columns=['SalePrice'])
indice = pd.DataFrame(ID_prueba,columns=['Id'])
resultado1 = pd.concat([indice, df_Estimador_final], axis=1)
resultado1.head(20)

,Id,SalePrice
0,1461,126125.429043
1,1462,153052.890990
2,1463,183451.719612
3,1464,184897.669124
4,1465,193655.222364
5,1466,179061.624581
6,1467,179885.714063
7,1468,165514.153954
8,1469,180502.889877
9,1470,127190.324264


In [18]:
df_Estimador_final = pd.DataFrame(Estimador_final_predic, columns=['SalePrice'])
resultado2 = pd.concat([prueba, df_Estimador_final], axis=1)
resultado2.head(20)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,126125.429043
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,153052.890990
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,183451.719612
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2010,WD,Normal,184897.669124
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,0,NaN,NaN,NaN,0,1,2010,WD,Normal,193655.222364
5,1466,60,RL,75.0,10000,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,179061.624581
6,1467,20,RL,NaN,7980,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,GdPrv,Shed,500,3,2010,WD,Normal,179885.714063
7,1468,60,RL,63.0,8402,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,165514.153954
8,1469,20,RL,85.0,10176,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2010,WD,Normal,180502.889877
9,1470,20,RL,70.0,8400,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,4,2010,WD,Normal,127190.324264


Exportamos el anterior DataFrame a un archivo CSV para que se pueda abrir en Excel

In [19]:
resultado2.to_csv('predicciones.csv',index=False)